### 기능 오류도 많고 허접함

In [4]:
# API Key를 환경변수로 관리하기 위한 설정 파일

from dotenv import load_dotenv

# API Key 정보로드
load_dotenv()

import os
os.environ["LANGCHAIN_PROJECT"] = "langchain_study"

In [1]:
import pprint
from typing import Any, Dict

import pandas as pd

In [21]:
df = pd.read_csv("titanic_passenger_list.csv", encoding='utf8')
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [22]:
df.loc[0]

pclass                                   1
survived                                 1
name         Allen, Miss. Elisabeth Walton
sex                                 female
age                                   29.0
sibsp                                    0
parch                                    0
ticket                               24160
fare                              211.3375
cabin                                   B5
embarked                                 S
boat                                     2
body                                   NaN
home.dest                     St Louis, MO
Name: 0, dtype: object

##### Model CASE #1. GPT-4o

In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0,
    max_tokens=2048,
    model="gpt-4o",
)

##### Model CASE #2. Ollama - Aya(4b Q)

In [34]:
from langchain_community.llms import Ollama

llm = Ollama(model="aya", temperature=0)

##### Parser 설정

In [23]:
from langchain.output_parsers import PandasDataFrameOutputParser

parser = PandasDataFrameOutputParser(dataframe=df)

##### Prompt 만들때 `parser.get_format_instructions()`로 format 지시사항을 추가

In [32]:
from langchain.prompts import PromptTemplate

# column, row, mean 등 간단한 명령만 가능
# filter 등 약간만 복잡해도 안됨
df_query = "Retrieve the passenges ages."
#df_query = "Retrieve the first row."
#df_query = "여자들의 평균 나이를 알려줘" # Error

# chatGPT가 자꾸 쌍따움표를 찍어서 출력해서 찍지말라고 명령 추가
prompt = PromptTemplate(
    template="Answer the user query. response without double quotations. \n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    }
)

In [ ]:
prompt

In [35]:
chain = prompt | llm | parser
response = chain.invoke({"query": df_query})
response

{'1': pclass                                     1
 survived                                   1
 name          Allison, Master. Hudson Trevor
 sex                                     male
 age                                     0.92
 sibsp                                      1
 parch                                      2
 ticket                                113781
 fare                                  151.55
 cabin                                C22 C26
 embarked                                   S
 boat                                      11
 body                                     NaN
 home.dest    Montreal, PQ / Chesterville, ON
 Name: 1, dtype: object}